<a href="https://colab.research.google.com/github/rafaelchourio/Deep-Learning-for-CV-Starter/blob/master/minivggnet_cifar10_lr_decay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
from importlib.machinery import SourceFileLoader
somemodule=SourceFileLoader("minivggnet","/content/drive/My Drive/Colab Notebooks/Workspace/nn/convolutionals/minivggnet.py").load_module()
from minivggnet import MiniVGGNet

In [0]:
def step_decay(epoch):
  # initialize the base initial learning rate, drop factor, and
  # epochs to drop every
  initAlpha = 0.01
  factor = 0.25
  dropEvery = 5

  # compute learning rate for the current epoch
  alpha = initAlpha * (factor ** np.floor((1 + epoch) / dropEvery))

  # return the learning rate
  return float(alpha)

In [4]:
# load the training and testing data, then scale it into the range [0, 1]
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()

trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

[INFO] loading CIFAR-10 data...
170500096/170498071 [==============================] - 2s 0us/step


In [0]:
# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [0]:
# initialize the label names for the CIFAR-10 dataset
labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [0]:
# define the set of callbacks to be passed to the model during
# training
callbacks = [LearningRateScheduler(step_decay)]

In [7]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=0.01, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

[INFO] compiling model...


In [8]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),
batch_size=64, epochs=40, callbacks=callbacks, verbose=1)

[INFO] training network...
Train on 50000 samples, validate on 10000 samples
Epoch 1/40
11648/50000 [=====>........................] - ETA: 4:11 - loss: 2.1177 - accuracy: 0.3385

KeyboardInterrupt: ignored

In [16]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1), target_names=labelNames))

[INFO] evaluating network...
              precision    recall  f1-score   support

    airplane       0.87      0.83      0.85      1000
  automobile       0.93      0.91      0.92      1000
        bird       0.80      0.72      0.76      1000
         cat       0.68      0.68      0.68      1000
        deer       0.78      0.84      0.81      1000
         dog       0.76      0.77      0.77      1000
        frog       0.85      0.88      0.87      1000
       horse       0.87      0.87      0.87      1000
        ship       0.91      0.91      0.91      1000
       truck       0.88      0.91      0.89      1000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



In [30]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure(figsize=(16,10))
plt.plot(np.arange(0, 40), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 40), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 40), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 40), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on CIFAR-10")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
#plt.savefig(args["output"])